In [1]:
import pandas
from pathlib import Path

import sys
base = "./../../../"
sys.path.append(base)

import pairs_flat_v2 as pairs
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import QuantileTransformer

# Load pandas
import pandas as pd

# Load numpy
import numpy as np

import helper
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
# Load matrix
oscope_gencounts = pandas.read_csv(Path(base + "data/GSE64016_H1andFUCCI_normalized_EC_human.csv"))

# Set index right
oscope_gencounts.set_index("Unnamed: 0", inplace=True)

# Subset sorted
oscope_gencounts_sorted = oscope_gencounts.iloc[:, [oscope_gencounts.columns.get_loc(c) for c in oscope_gencounts.columns if "G1_" in c or "G2_" in c or "S_" in c]]

oscope_gencounts_sorted.head(10)

,G2_Exp1.059,G2_Exp1.069,G2_Exp1.075,G2_Exp1.063,G2_Exp1.029,G2_Exp1.076,G2_Exp1.013,G2_Exp1.037,G2_Exp1.057,G2_Exp1.018,...,G1_Exp1.008,G1_Exp1.055,G1_Exp1.050,G1_Exp1.076,G1_Exp1.011,G1_Exp1.063,G1_Exp1.083,G1_Exp1.030,G1_Exp1.018,G1_Exp1.046
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
MKL2,26.258941,7.349801,0.590667,9.526098,2.653238,132.288290,26.872890,0.000000,21.962493,577.589300,...,3.887628,84.337868,69.192927,1.126491,1.137330,0.000000,36.741767,11.218839,152.792860,123.041274
CD109,2.234584,6.094681,76.627285,1.011232,2.012671,10.856459,54.251989,164.865603,1.601655,6.700829,...,4.956726,7.208501,4.299846,7.626347,5.936864,2.797575,149.063512,2.804710,15.996667,7.077119
ABTB1,0.514881,0.000000,0.000000,1.831942,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MAST2,0.514881,0.565369,67.336087,0.000000,23.879144,0.000000,63.744994,0.000000,0.000000,118.848390,...,0.971907,15.069427,0.000000,60.830538,22.746604,0.000000,0.720427,0.000000,5.589983,33.230125
KAT5,3.980032,0.576677,0.000000,338.473236,0.000000,0.000000,61.407678,2.792845,285.432055,0.000000,...,0.000000,96.041736,0.000000,0.000000,0.000000,0.000000,1.030210,55.091510,4.826018,0.000000
WWC2,90.104208,642.694883,484.164183,74.743227,362.341379,341.330246,692.058151,253.949365,327.808953,686.495120,...,169.111820,233.122597,656.509085,412.712674,1075.186455,661.889223,104.641994,322.527589,47.514853,67.124853
CD163,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MYL2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
UBE2Z,372.264249,140.211592,12.994683,12.090816,58.750276,94.365647,309.350706,130.066760,140.345685,20.347584,...,138.010796,58.877380,538.955243,5.632457,71.651801,0.656708,2.881707,610.984967,0.000000,3.215239


In [3]:
# Get annotation
is_G1 = [oscope_gencounts_sorted.columns.get_loc(c) for c in oscope_gencounts_sorted.columns if "G1_" in c]
is_S = [oscope_gencounts_sorted.columns.get_loc(c) for c in oscope_gencounts_sorted.columns if "S_" in c]
is_G2M = [oscope_gencounts_sorted.columns.get_loc(c) for c in oscope_gencounts_sorted.columns if "G2_" in c]

annotation = {
    "G1": list(is_G1),
    "S": list(is_S),
    "G2M": list(is_G2M)
}

go_0007049 = [line.replace("\n","").replace("\r","") for line in open(base + "data/go_0007049_homoSapiens.csv", "r")]
cycle_base = [line.split("\t")[0] for i, line in enumerate(open(base + "data/cyclebase_top1000_genes.tsv", "r")) if 0 < i]
cycle_genes = np.unique(np.concatenate((go_0007049, cycle_base),0))

oscope_marker_pairs = pairs.sandbag(x=oscope_gencounts_sorted, phases=annotation, subset_genes=list(cycle_genes), fraction=0.6, processes=0, verbose=True)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Removed 16689 genes that were not in 'subset_genes'. 2395 genes remaining.
[__set_matrix] Removed 61 genes that were not expressed in any samples. 2334 genes remaining.
[__set_matrix] Removed 0 samples that were not annotated in 'phases'. 247 samples remaining.
[__set_matrix] Matrix truncation done. Working with 2334 genes for 247 samples.
[sandbag] Identifying marker pairs...Processing in parallel with 15 processes...
 Done!
[sandbag] Identified 8146 marker pairs (phase: count): {'G1': 2575, 'S': 4101, 'G2M': 1470}


# Random Forest with pairs as Markers

In [4]:
import pandas
oscope_pairs_features = pandas.DataFrame(index=oscope_gencounts_sorted.columns)

In [5]:
from tqdm import tqdm_notebook as tqdm
classes = ["G1", "S", "G2M"]

label = [classes.index(i[:i.index('_')].replace("G2", "G2M")) for i in oscope_pairs_features.index]

oscope_pairs_features = oscope_pairs_features.assign(label=label)

for phase, marker in tqdm(oscope_marker_pairs.items()):
    for pair in tqdm(marker):
        name = "{}_{}-{}".format(phase, *pair)
        
        values = [1 if oscope_gencounts_sorted.loc[pair[0],sample] > oscope_gencounts_sorted.loc[pair[1],sample]
                  else 0 for sample in oscope_gencounts_sorted.columns]
        kwargs = {name: values}
        
        oscope_pairs_features = oscope_pairs_features.assign(**kwargs)

oscope_pairs_features.head()


HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

HBox(children=(IntProgress(value=0, max=2575), HTML(value='')))

HBox(children=(IntProgress(value=0, max=4101), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1470), HTML(value='')))

,label,G1_AASDHPPT-ANKRD17,G1_AASDHPPT-BIRC6,G1_AASDHPPT-BRCA2,G1_AASDHPPT-CDK1,G1_AASDHPPT-KHDRBS1,G1_AASDHPPT-KIF5B,G1_AASDHPPT-NCAPG,G1_AASDHPPT-NUP205,G1_AASDHPPT-POGZ,...,G2M_UBE2C-WSB1,G2M_UBE2C-WTAP,G2M_UBE2C-YWHAH,G2M_UBE2C-ZFP36L2,G2M_UBE2C-ZFR,G2M_UBE2C-ZRANB2,G2M_UBE2C-ZWILCH,G2M_UBE2C-ZWINT,G2M_UBE2G1-WTAP,G2M_XPO1-ZNF207
G2_Exp1.059,2,1,1,1,0,0,1,1,0,0,...,0,0,1,1,1,1,0,0,1,1
G2_Exp1.069,2,0,0,1,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
G2_Exp1.075,2,0,0,0,0,0,0,0,1,0,...,1,1,1,1,1,1,1,0,1,1
G2_Exp1.063,2,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,0,1,0,0
G2_Exp1.029,2,1,1,1,0,1,1,0,0,0,...,1,1,1,1,1,1,1,1,1,0


## Internal validation on oscope dataset

In [6]:
np.random.seed(0)

oscope_pairs_features['is_train'] = np.random.uniform(0, 1, len(oscope_pairs_features)) <= .75
train, test = oscope_pairs_features[oscope_pairs_features['is_train']==True], oscope_pairs_features[oscope_pairs_features['is_train']==False]
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 196
Number of observations in the test data: 51


In [7]:
features = oscope_pairs_features.columns[1:-1]
features

Index(['G1_AASDHPPT-ANKRD17', 'G1_AASDHPPT-BIRC6', 'G1_AASDHPPT-BRCA2',
       'G1_AASDHPPT-CDK1', 'G1_AASDHPPT-KHDRBS1', 'G1_AASDHPPT-KIF5B',
       'G1_AASDHPPT-NCAPG', 'G1_AASDHPPT-NUP205', 'G1_AASDHPPT-POGZ',
       'G1_AASDHPPT-RBBP8',
       ...
       'G2M_UBE2C-WSB1', 'G2M_UBE2C-WTAP', 'G2M_UBE2C-YWHAH',
       'G2M_UBE2C-ZFP36L2', 'G2M_UBE2C-ZFR', 'G2M_UBE2C-ZRANB2',
       'G2M_UBE2C-ZWILCH', 'G2M_UBE2C-ZWINT', 'G2M_UBE2G1-WTAP',
       'G2M_XPO1-ZNF207'],
      dtype='object', length=8146)

In [8]:
y = train['label'].values
y

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [9]:
clf = RandomForestClassifier(n_jobs=10, random_state=0)
clf.fit(train[features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=10,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [10]:
clf.predict(test[features])

array([1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [11]:
classes = np.array(["G1", "S", "G2M"])
preds = classes[clf.predict(test[features])]
labels = classes[test['label']]
pd.crosstab(labels, preds, rownames=['Actual Phase'], colnames=['Predicted Phase'])

Predicted Phase,G1,G2M,S
Actual Phase,,,
G1,16,0,1
G2M,0,15,3
S,1,0,15


In [12]:
feature_importance = list(zip(train[features], clf.feature_importances_))
feature_importance = sorted(feature_importance, key=lambda x: x[1], reverse=True)
feature_importance[0:25]

[('G1_UBE2L3-HIST1H4C', 0.04370413323721306),
 ('G1_TOB1-CDK1', 0.03769776758120596),
 ('G2M_UBE2C-PHB2', 0.03331654672680516),
 ('G1_CLIC1-HIST1H4C', 0.03293357957678247),
 ('G1_PARD6B-HIST1H4C', 0.031531080843114304),
 ('S_MIF-CKS2', 0.03148547392591753),
 ('G1_NOP16-CDK1', 0.02733169836479212),
 ('G1_PSMC3-CDK1', 0.027316999679852204),
 ('G1_UBE2D3-HIST1H4C', 0.02539166683187748),
 ('G1_MAPK6-HIST1H4C', 0.02538356019837502),
 ('G1_GMNN-HIST1H4C', 0.025130611457745033),
 ('G1_CCND2-HIST1H4C', 0.02502559321009979),
 ('S_RIF1-CCNB1', 0.02500212246673031),
 ('G2M_UBE2C-CALR', 0.0238221278556194),
 ('G2M_TOP2A-GMNN', 0.02232224084681417),
 ('S_PSMA7-CCNB1', 0.021950247968766483),
 ('S_DNAJB6-CKS2', 0.021511132671198297),
 ('G2M_CDK1-CCAR1', 0.018314062484027012),
 ('S_SEPT7-CCNB1', 0.018138523959549162),
 ('G1_PCBP1-HIST1H4C', 0.017890357767531753),
 ('S_HCFC1-ARHGAP11A', 0.01740390568377323),
 ('G1_AURKB-UBE2C', 0.016065933082841294),
 ('S_HAUS6-CKS2', 0.01577328073987385),
 ('G1_PSMB6-

# Cyclone for reference

In [13]:
oscope_cyclone = pairs.cyclone(x=oscope_gencounts_sorted, marker_pairs=oscope_marker_pairs, subset_samples=test.index.tolist(), verbose=True)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Removed 196 samples that were not in 'subset_samples'. 51 samples remaining.
[__set_matrix] Matrix truncation done. Working with 19084 genes for 51 samples.
[cyclone] Preparing marker pairs, where at least one gene was not present in 'x'... Done!
[cyclone] Removed 0 marker pairs. 8146 marker pairs remaining.
[cyclone] Calculating scores and predicting cell cycle phase... Done!
[cyclone] Calculated scores and prediction (phase: count): G2M: 17, S: 17, G1: 17


In [14]:
preds = np.array([pred for sample, pred in oscope_cyclone["prediction"].items()])
pd.crosstab(labels, preds, rownames=['Actual Phase'], colnames=['Predicted Phase'])

Predicted Phase,G1,G2M,S
Actual Phase,,,
G1,17,0,0
G2M,0,17,1
S,0,0,16


# Prediction on a bulk sample

In [15]:
gencounts_EMATB6142 = pandas.read_csv(Path(base + "data/E-MTAB-6142_human.csv"), sep=';')
gencounts_EMATB6142.set_index("Gene_ID", inplace=True)
gene_map = {}

with open(base + "data/biomart_human-genes.txt", "r") as f:
    for line in f:
        info = line.split(",")
        gene_map[info[0].replace("\n","").replace("\r","")] = info[1].replace("\n","")

index_list = gencounts_EMATB6142.index.tolist()

for idx, i in enumerate(index_list):
    try:
        if "." in i:
            index_list[idx] = gene_map[i[:i.index(".")]]
        else:
            index_list[idx] = gene_map[i] 
    except KeyError:
        pass

gencounts_EMATB6142.index = index_list
#gencounts_EMATB6142 = gencounts_EMATB6142[~gencounts_EMATB6142.index.duplicated(keep=False)]
gencounts_EMATB6142.head(10)

,S1_G1,S2_G1,S3_G1,S4_G1,S5_G1,S6_G1,S7_G1,S8_G1,S9_G1,S10_G1,...,S87_G2M,S88_G2M,S89_G2M,S90_G2M,S91_G2M,S92_G2M,S93_G2M,S94_G2M,S95_G2M,S96_G2M
TSPAN6,360,5,437,136,328,253,1101,39,157,253,...,391,429,148,397,424,317,403,280,470,725
TNMD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DPM1,111,421,70,179,93,57,60,35,174,91,...,63,228,173,115,87,308,92,179,164,104
SCYL3,2,5,0,0,1,1,3,0,5,1,...,3,0,2,1,2,0,0,0,38,1
C1orf112,179,448,0,0,135,47,0,0,0,159,...,151,68,147,84,151,11,0,0,169,77
FGR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CFH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,34,0,0,0
FUCA2,293,0,0,389,65,229,106,95,112,205,...,175,110,375,46,55,291,283,224,293,93
GCLC,0,0,0,0,208,0,0,0,0,0,...,0,0,0,0,0,0,40,0,0,0
NFYA,0,118,0,0,0,0,0,60,0,0,...,10,0,0,98,108,149,60,125,46,2


In [16]:
x = gencounts_EMATB6142.T.values

X_std = QuantileTransformer().fit_transform(x.astype(float))

gencounts_EMATB6142_Qnorm = pandas.DataFrame(X_std.T, index=gencounts_EMATB6142.index, columns=gencounts_EMATB6142.columns)

gencounts_EMATB6142_Qnorm.head(10)

,S1_G1,S2_G1,S3_G1,S4_G1,S5_G1,S6_G1,S7_G1,S8_G1,S9_G1,S10_G1,...,S87_G2M,S88_G2M,S89_G2M,S90_G2M,S91_G2M,S92_G2M,S93_G2M,S94_G2M,S95_G2M,S96_G2M
TSPAN6,3.473684e-01,1.000000e-07,5.264640e-01,3.131861e-02,3.050698e-01,1.526527e-01,9.789371e-01,1.029057e-02,5.297821e-02,1.526527e-01,...,4.210878e-01,4.945908e-01,4.212460e-02,4.369369e-01,4.735360e-01,2.738559e-01,4.525139e-01,2.210190e-01,6.156156e-01,8.947715e-01
TNMD,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07
DPM1,3.683954e-01,9.890796e-01,1.736737e-01,6.896897e-01,2.945035e-01,8.958959e-02,1.052632e-01,4.754755e-02,6.416416e-01,2.631579e-01,...,1.157895e-01,8.105044e-01,6.210526e-01,3.895449e-01,2.316756e-01,9.473313e-01,2.787788e-01,6.896897e-01,5.735736e-01,3.313313e-01
SCYL3,7.157157e-01,8.683684e-01,1.000000e-07,1.000000e-07,5.470470e-01,5.470470e-01,8.108108e-01,1.000000e-07,8.683684e-01,5.470470e-01,...,8.108108e-01,1.000000e-07,7.157157e-01,5.470470e-01,7.157157e-01,1.000000e-07,1.000000e-07,1.000000e-07,9.314314e-01,5.470470e-01
C1orf112,9.158360e-01,9.999999e-01,1.000000e-07,1.000000e-07,8.213680e-01,4.843728e-01,1.000000e-07,1.000000e-07,1.000000e-07,8.946219e-01,...,8.683684e-01,5.685140e-01,8.426397e-01,6.526420e-01,8.683684e-01,3.579496e-01,1.000000e-07,1.000000e-07,9.052632e-01,6.314140e-01
FGR,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07
CFH,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,9.894737e-01,1.000000e-07,1.000000e-07,1.000000e-07
FUCA2,9.104104e-01,1.000000e-07,1.000000e-07,9.890842e-01,2.632633e-01,7.893646e-01,4.630937e-01,4.159159e-01,4.894895e-01,7.050920e-01,...,6.315873e-01,4.738216e-01,9.789474e-01,1.683129e-01,2.367367e-01,8.947837e-01,8.738128e-01,7.789999e-01,9.104104e-01,3.948949e-01
GCLC,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,9.789681e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,9.367956e-01,1.000000e-07,1.000000e-07,1.000000e-07
NFYA,1.000000e-07,8.368368e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,6.051051e-01,1.000000e-07,1.000000e-07,...,5.051710e-01,1.000000e-07,1.000000e-07,7.266548e-01,7.788708e-01,9.259958e-01,6.051051e-01,8.633743e-01,5.788832e-01,4.579580e-01


In [17]:
EMATB6142_prediction = pairs.cyclone(gencounts_EMATB6142_Qnorm, oscope_marker_pairs, verbose=True)

[__set_matrix] Original Matrix 'x' has shape 59838 x 96
[__set_matrix] Matrix truncation done. Working with 59838 genes for 96 samples.
[cyclone] Preparing marker pairs, where at least one gene was not present in 'x'... Done!
[cyclone] Removed 412 marker pairs. 8146 marker pairs remaining.
[cyclone] Calculating scores and predicting cell cycle phase... Done!
[cyclone] Calculated scores and prediction (phase: count): G1: 32, S: 14, G2M: 50


In [18]:
EMATB6142_prediction_table = helper.get_prediction_table(EMATB6142_prediction)
helper.DataTable(EMATB6142_prediction_table)

,G1,G2M,S,G1_norm,G2M_norm,S_norm,prediction
sample,,,,,,,
S1_G1,0.999,0.000,1.000,0.499750,0.000000,0.500250,G1
S2_G1,0.999,0.194,0.000,0.837385,0.162615,0.000000,G1
S3_G1,0.967,0.304,0.000,0.760818,0.239182,0.000000,G1
S4_G1,0.991,0.545,0.000,0.645182,0.354818,0.000000,G1
S5_G1,0.298,0.248,0.988,0.194263,0.161669,0.644068,S
S6_G1,0.666,0.040,1.000,0.390387,0.023447,0.586166,G1
S7_G1,0.949,0.938,0.000,0.502915,0.497085,0.000000,G1
S8_G1,0.994,0.000,0.417,0.704465,0.000000,0.295535,G1
S9_G1,0.999,0.898,0.000,0.526621,0.473379,0.000000,G1


In [19]:
EMATB6142_labels = list(['G1'] * 32) + list(['S'] * 32) + list(['G2M'] * 32)

In [20]:
EMATB6142_evaluation = helper.evaluate_prediction(EMATB6142_prediction_table, EMATB6142_labels)

F1 Score: G1: 0.84375, S: 0.4782608695652174, G2M: 0.6585365853658537
Reacall: G1: 0.84375, S: 0.34375, G2M: 0.84375 
Precision: G1: 0.84375, S: 0.7857142857142857, G2M: 0.54 


In [21]:
iplot(helper.plot_evaluation(*EMATB6142_evaluation, xaxis=["G1","S","G2M"], xaxislbl="Phase", average=True, title="Pairs prediction scores for EMATB6142"), image="svg")

In [22]:
clf = RandomForestClassifier(n_jobs=10, random_state=0)
y = oscope_pairs_features['label'].values
clf.fit(oscope_pairs_features[features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=10,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [23]:
gencounts_EMATB6142_Qnorm.drop_duplicates(keep=False, inplace=True)

EMATB6142_pairs_features = pandas.DataFrame(index=gencounts_EMATB6142_Qnorm.columns)

for phase, marker in tqdm(oscope_marker_pairs.items()):
    for pair in tqdm(marker):
        name = "{}_{}-{}".format(phase, *pair)
        
        values = [1 if pair[0] in gencounts_EMATB6142_Qnorm.index and pair[1] in gencounts_EMATB6142_Qnorm.index and gencounts_EMATB6142_Qnorm.loc[pair[0],sample] > gencounts_EMATB6142_Qnorm.loc[pair[1],sample]
                  else 0 for sample in gencounts_EMATB6142_Qnorm.columns]
        kwargs = {name: values}
        
        EMATB6142_pairs_features = EMATB6142_pairs_features.assign(**kwargs)



HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

HBox(children=(IntProgress(value=0, max=2575), HTML(value='')))

HBox(children=(IntProgress(value=0, max=4101), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1470), HTML(value='')))

In [24]:
EMATB6142_rf_prediction = clf.predict(EMATB6142_pairs_features[features])
EMATB6142_rf_prediction

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 1, 0, 2, 2, 2, 1, 1, 1, 2,
       1, 0, 1, 2, 1, 0, 1, 2, 2, 2, 0, 1, 1, 1, 2, 2, 0, 1, 1, 0, 2, 0,
       2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 1, 2, 0, 2, 0, 0, 0, 0, 0,
       0, 2, 2, 0, 1, 0, 2, 0], dtype=int64)

In [25]:
classes = np.array(["G1", "S", "G2M"])
preds = classes[EMATB6142_rf_prediction]

EMATB6142_rf_prediction_table = pandas.DataFrame(index=gencounts_EMATB6142_Qnorm.columns)
EMATB6142_rf_prediction_table = EMATB6142_rf_prediction_table.assign(prediction=preds)

helper.DataTable(EMATB6142_rf_prediction_table)

,prediction
S1_G1,G1
S2_G1,G1
S3_G1,G1
S4_G1,G1
S5_G1,S
S6_G1,S
S7_G1,G1
S8_G1,G1
S9_G1,G1
S10_G1,G1


In [26]:
EMATB6142_rf_evaluation = helper.evaluate_prediction(EMATB6142_rf_prediction_table, EMATB6142_labels)
iplot(helper.plot_evaluation(*EMATB6142_rf_evaluation, xaxis=["G1","S","G2M"], xaxislbl="Phase", average=True, title="Random Forest with Pairs prediction scores for EMATB6142"), image="svg")

F1 Score: G1: 0.5925925925925924, S: 0.5555555555555556, G2M: 0.3859649122807018
Reacall: G1: 0.75, S: 0.46875, G2M: 0.34375 
Precision: G1: 0.4897959183673469, S: 0.6818181818181818, G2M: 0.44 
